In [33]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


### Load the dataset and prepare the labels and sequences

In [1]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

# Change working directory to Project folder
%cd "/content/drive/MyDrive/Machine_Learning_(CS-433)/Project_2"
# I assume this directory contains two folders: data and models, containing the data and models respectively.
file_path = 'data/dataset.txt'

# Load dataset
df = pd.read_csv(file_path, sep='\t')

# Drop sequences that don't natively fit in splicebert
df = df.drop(df[df["IVS_SIZE"] > 1024].index)
# This should be improved at a later time

#Pick just a random subset to make the processing to come run fast.
df = df.sample(n = 10)


Mounted at /content/drive
/content/drive/MyDrive/Machine_Learning_(CS-433)/Project_2


In [2]:
df.columns.values

array(['CHR', 'START', 'END', 'STRAND', 'GENE', 'TRANSCRIPT', 'IVS',
       'IVS_SIZE', 'BP_POS', 'BP_ACC_DIST', 'BP_ACC_SEQ', 'IVS_SEQ'],
      dtype=object)

In [3]:
df

,CHR,START,END,STRAND,GENE,TRANSCRIPT,IVS,IVS_SIZE,BP_POS,BP_ACC_DIST,BP_ACC_SEQ,IVS_SEQ
109595,chr12,56339525,56339690,+,IL23A,ENST00000228534,IVS2,166,56339670,-21,ATTGCTTTCTTTTCTCCCTAG,GTACCACTGGGATGTGGCTGGGCAATGAAGGAGAGGGGACTGAGAA...
132511,chr16,18836533,18837252,-,SMG1,ENST00000446231,IVS46,720,18836557,-25,ATTTTGTTTGTGTCCCCCTGATTAG,GTTTAAAACAGTGTTGAATTCTTCTAAATATGTGCCATTAATTAGA...
13964,chr1,182852345,182853305,+,DHX9,ENST00000367549,IVS4,961,182853302,-4,ACAG,GTAAGGGACTTGAATCCATGCACGTGGTGGAGAATAAGATATACAC...
11716,chr1,156137233,156137653,+,LMNA,ENST00000368300,IVS9,421,156137631,-23,ACCTGGTTCCATGTCCCCACCAG,GTAAGTAGGCCTGGGCCTGGCTGCTTGCTGGACGAGGCTCCCCCTG...
41270,chr3,180603324,180604224,+,TTC14,ENST00000296015,IVS3,901,180604207,-18,AAATACTTCTCATTACAG,GTAAGTTATTTTTGTTACTTGGATTGCTTCTGTTTTTGTTAACGCA...
37589,chr3,108375801,108376492,+,HHLA2,ENST00000467761,IVS8,692,108376470,-23,AGTTCTCTTTTTTTTTCCTGTAG,GTCAGCCTCCCATGTCTGAGAGAAGAATGGATTCTGGTTCTGGAGG...
145153,chr17,57849411,57849966,-,MRPS23,ENST00000313608,IVS1,556,57849435,-25,AGTTTCGTTCTGTTCCCTCCCTAAG,GTGTGTGCTCCCAGCCGTGGTGGGCTGAGGGTGCCTCCCCCAGGCT...
161001,chr19,55175340,55175708,-,SYT5,ENST00000354308,IVS5,369,55175364,-25,ACTGCGGATTCTCTGTGCACTCCAG,GTGAGCTCCTGCCTTCAGGCCCCGTGTTCCATAGCCTGTGCCCAGT...
99688,chr11,64317148,64317263,-,TRMT112,ENST00000544844,IVS2,116,64317151,-4,ACAG,GTGAGGATCCTTACCCCGCCCCAGCATATCCCGGGGAATGCGGGGC...
20957,chr2,48723534,48723621,-,LHCGR,ENST00000294954,IVS5,88,48723558,-25,AATATTTCTCATTTCTTTCCTCAAG,GTAAGTACCAGGCTGATTGCTATGCATAACAGTTTCCTATTTGCAG...


In [4]:
df["IVS_SEQ"]

,IVS_SEQ
109595,GTACCACTGGGATGTGGCTGGGCAATGAAGGAGAGGGGACTGAGAA...
132511,GTTTAAAACAGTGTTGAATTCTTCTAAATATGTGCCATTAATTAGA...
13964,GTAAGGGACTTGAATCCATGCACGTGGTGGAGAATAAGATATACAC...
11716,GTAAGTAGGCCTGGGCCTGGCTGCTTGCTGGACGAGGCTCCCCCTG...
41270,GTAAGTTATTTTTGTTACTTGGATTGCTTCTGTTTTTGTTAACGCA...
37589,GTCAGCCTCCCATGTCTGAGAGAAGAATGGATTCTGGTTCTGGAGG...
145153,GTGTGTGCTCCCAGCCGTGGTGGGCTGAGGGTGCCTCCCCCAGGCT...
161001,GTGAGCTCCTGCCTTCAGGCCCCGTGTTCCATAGCCTGTGCCCAGT...
99688,GTGAGGATCCTTACCCCGCCCCAGCATATCCCGGGGAATGCGGGGC...
20957,GTAAGTACCAGGCTGATTGCTATGCATAACAGTTTCCTATTTGCAG...


In [29]:
#Create training data
input_sequences = df["IVS_SEQ"].to_list()
input_labels = []

for index, sample in df.iterrows():
    label_sequence = (sample["BP_POS"]-sample["START"])*[0] + [1] + (sample["END"]-sample["BP_POS"])*[0]
    input_labels.append(label_sequence)

In [6]:
input_sequences

['GTACCACTGGGATGTGGCTGGGCAATGAAGGAGAGGGGACTGAGAACATGGCTGGGTACCATGGTAAACCAGAAGTTGTGTCTGAAAATAGTAAGAAACTGGGTGAGTCTTCAGTGAATGGAGTAGGAAGAGGGTGTCCTCTTTCATTGCTTTCTTTTCTCCCTAG',
 'GTTTAAAACAGTGTTGAATTCTTCTAAATATGTGCCATTAATTAGAATTTTCATATTCATGTAAACATCAATATGGATTAGAATTATTGTAACAAATCACATAAATGATAAAAGCTGTCTCAGTGACTGCAACTTCAAAGTGTTAGTAAACAAAAATTCACTGAAATTGTCTGCTTTTATTATATTTAACTTAAATGTTTGAATGACTAATTATGGGAGATAGGTGGGTGTAGTGGTTAGAACACAAAACTTGGAATCAGAAAATCTGGGTTTTGGTTTTCTCATTACCATTATGTATCTTGAGTCCTTACTCATTTGTACAATGGCAGCTACTTCATCTACCTCACTAAGTATTAAATAAGATGATTAACTGAAAGAACCTTATACACAGCAGGTGCTCAGTAAAAGGGTAGTTGAATATGAAAGTGTCAAGCATTGTGTGAGGCTCTCAGATTACAAAGATCTGTCAAGCATCAGTCTCTAACCTTCAAGGAACCTAAAACTTGGCAGATGTGGCAGTTTTAATTAAAATGTCATGAGCATGGTTGAGGGAATCACAGGGTCAGGAAAAACAGAGAAGTAAGGGACATTTGAGGAGGGCCCTCAAGGTGAACAAGGAGCGTACCAGACAGTCCATGTGTGAGCACATTCCAGGGTAGGAAAGTATGTGGAAGAAAACAGCAATAAATCTTTTGATTTTGTTTGTGTCCCCCTGATTAG',
 'GTAAGGGACTTGAATCCATGCACGTGGTGGAGAATAAGATATACACAATCTTGATCACCATCTCTGTTTCTCGTTTTGGGTAGAAAGAACTGCAGTAAACTC

In [7]:
input_labels

['0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000100000000000000000000',
 '000000000000000000000000100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000',
 '000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

### Load the splicebert model and tokenizer

In [8]:
SPLICEBERT_PATH = "models/SpliceBERT.1024nt"  # set the path to the folder of pre-trained SpliceBERT
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForMaskedLM, AutoModelForTokenClassification

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(SPLICEBERT_PATH)

# finetuning SpliceBERT for token classification tasks
model = AutoModelForTokenClassification.from_pretrained(SPLICEBERT_PATH, num_labels = 2) # We want binary classification on tokens so num_labels = 2

# use huggerface's official API to use SpliceBERT
# get nucleotide embeddings (hidden states)
#model = AutoModel.from_pretrained(SPLICEBERT_PATH) # load model
#last_hidden_state = model(input_ids).last_hidden_state # get hidden states from last layer
#hiddens_states = model(input_ids, output_hidden_states=True).hidden_states # hidden states from the embedding layer (nn.Embedding) and the 6 transformer encoder layers

# get nucleotide type logits in masked language modeling
#model = AutoModelForMaskedLM.from_pretrained(SPLICEBERT_PATH) # load model
#logits = model(input_ids).logits # shape: (batch_size, sequence_length, vocab_size)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at models/SpliceBERT.1024nt and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Show a case of inference

In [70]:
# prepare a test input to show how to do inference
seq = "ACGUACGuacguaCGuGCAUGUCAUGC"  ## WARNING: this is just a demo. SpliceBERT may not work on sequences shorter than 64nt as it was trained on sequences of 64-1024nt in length
seq = ' '.join(list(seq.upper().replace("U", "T"))) # U -> T and add whitespace
input_ids = tokenizer.encode(seq) # N -> 5, A -> 6, C -> 7, G -> 8, T(U) -> 9. NOTE: a [CLS] and a [SEP] token will be added to the start and the end of seq
print(input_ids)
input_ids = torch.as_tensor(input_ids) # convert python list to Tensor
input_ids = input_ids.unsqueeze(0) # add batch dimension, shape: (batch_size, sequence_length)

[2, 6, 7, 8, 9, 6, 7, 8, 9, 6, 7, 8, 9, 6, 7, 8, 9, 8, 7, 6, 9, 8, 9, 7, 6, 9, 8, 7, 3]


In [10]:
#infer with the (untrained) model
logit = model(input_ids, output_hidden_states=False).logits

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


In [11]:
#Define a function for making the output interpretable
def logittobinary(logits):
  """
  This is just a ghetto function for converting binary logits into binary strings
  """
  output_list = []
  for j in range(len(logits)):
    binary = ""
    for i in range(len(logits[j,:,1])):
      if logits[j,i,0] > logits[j,i,1]:
        binary += "1"
      else:
        binary += "0"
    output_list.append(binary)
  return output_list



In [12]:
logit = model(input_ids, output_hidden_states=False).logits

In [68]:
print(logittobinary(logit))

['11111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111

The model is not yet finetuned so this output is just gibberish at the moment. Notice that for each token the likelyhood of a branchpoint is about the same as not a branchpoint, which should definately not be the case once the model has been trained.

### Tokenize the training data

In [19]:
#Tokenize the input data
input_sequences = [' '.join(list(seq.upper().replace("U", "T"))) for seq in input_sequences]
print(input_sequences)

['G   T   A   C   C   A   C   T   G   G   G   A   T   G   T   G   G   C   T   G   G   G   C   A   A   T   G   A   A   G   G   A   G   A   G   G   G   G   A   C   T   G   A   G   A   A   C   A   T   G   G   C   T   G   G   G   T   A   C   C   A   T   G   G   T   A   A   A   C   C   A   G   A   A   G   T   T   G   T   G   T   C   T   G   A   A   A   A   T   A   G   T   A   A   G   A   A   A   C   T   G   G   G   T   G   A   G   T   C   T   T   C   A   G   T   G   A   A   T   G   G   A   G   T   A   G   G   A   A   G   A   G   G   G   T   G   T   C   C   T   C   T   T   T   C   A   T   T   G   C   T   T   T   C   T   T   T   T   C   T   C   C   C   T   A   G', 'G   T   T   T   A   A   A   A   C   A   G   T   G   T   T   G   A   A   T   T   C   T   T   C   T   A   A   A   T   A   T   G   T   G   C   C   A   T   T   A   A   T   T   A   G   A   A   T   T   T   T   C   A   T   A   T   T   C   A   T   G   T   A   A   A   C   A   T   C   A   A   T   A   T   G   G   A   T   T   A   G   A   A   T

In [20]:
[print(len(input_sequence)) for input_sequence in input_sequences]

661
2877
3841
1681
3601
2765
2221
1473
461
349


[None, None, None, None, None, None, None, None, None, None]

In [53]:
#packaged for use in dataset class
input_ids = tokenizer(input_sequences, padding = 'max_length', padding_side = 'left', max_length = 1024) #, padding = 'max_length', padding_side = 'left', max_length = 1024) #We can do padding at this step but i think we use data_collator later for that

In [54]:
# Build the dataset structure we will pass for training
from datasets import Dataset

train_dataset = Dataset.from_dict(input_ids)
train_dataset = train_dataset.add_column("labels", input_labels)

In [55]:
#Set up the collator (I think it does padding)
from transformers import DataCollatorForTokenClassification, TrainingArguments, Trainer
data_collator = DataCollatorForTokenClassification(tokenizer)

In [56]:
#Define model training parameters
model_name = SPLICEBERT_PATH.split("/")[-1]
batch_size = 4

args = TrainingArguments(
    f"{model_name}-finetuned-secondary-structure",
    eval_strategy = "no",
    save_strategy = "epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.001,
    #load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    #push_to_hub=True,
)

In [57]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()


<ipython-input-57-ffb7fee7e86c>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss


TrainOutput(global_step=3, training_loss=0.3988550901412964, metrics={'train_runtime': 95.3296, 'train_samples_per_second': 0.105, 'train_steps_per_second': 0.031, 'total_flos': 1162220789760.0, 'train_loss': 0.3988550901412964, 'epoch': 1.0})

In [66]:
trained_model = AutoModelForTokenClassification.from_pretrained(f"{model_name}-finetuned-secondary-structure/checkpoint-3")

In [71]:

logit = trained_model(input_ids, output_hidden_states=False).logits

In [72]:
print(logittobinary(logit))

['11111111111111111111111111111']


In [73]:
input_labels

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,